In [2]:
import pandas as pd
import numpy as np
import h5py

from selfpeptide.utils.data_utils import Self_NonSelf_PeptideDataset

In [3]:
hdf5_file = "../processed_data/pre_tokenized_peptides_dataset.hdf5"
dset = Self_NonSelf_PeptideDataset(hdf5_file, gen_size=500000, val_size=0)

In [4]:
dset.peptides

tensor([[ 1, 20,  2,  ..., 22, 22, 22],
        [ 8,  9,  7,  ..., 20,  0,  8],
        [16, 11,  7,  ..., 15,  1, 11],
        ...,
        [12, 15,  5,  ..., 12, 22, 22],
        [12,  9,  9,  ..., 22, 22, 22],
        [ 5, 18, 16,  ..., 20, 22, 22]])